<a target="_blank" href="https://colab.research.google.com/github/cyneuro/Neural-Networks-Machine-Learning/blob/main/stats/Maximum_Likelihood_Estimation.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Maximum Likelihood Estimation

This notebook assumes the data is a standard continuous normal distribution. The number of random samples, mean, and standard deviation of the distribution can be set in lines 7-9 in code cell 1.

The function `gaussian` calculates the negative log likelihood given the random samples and the initial predicted mean and standard deviations defined in initParams.

The key to the whole program is the scipy function `minimize`. It is a regular optimization function and more information about it can be found [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html).

**Note:** the method can be a lot of different options, all different mathematical optimization techniques.


#### Questions:
1. If everything is working, we would expect more accurate $\mu^*$ and $\sigma^*$ predictions with more random samples. Think about why this is.
2. What happens when the initial parameter guesses are extremely off? What about when they're extremely close? How does this impact the number of samples?
3. Break down each line in the `gaussian` function. Knowing what you know about MLE, what do you think the function `stats.norm.logpdf` does?

In [ ]:
from scipy import stats
import numpy as np
from scipy.optimize import minimize
np.random.seed(1)

# Define the true parameters for Logistic distribution
samples = 100
mu = 2
s = 1.5

# Generate sample data from Logistic distribution
sample_data = np.random.logistic(loc=mu, scale=s, size=samples)


# Negative log likelihood function for Logistic distribution
def logistic_nll(params):
    mean, scale = params[0], params[1]

    if scale <= 0:  # Ensure scale parameter remains positive
        return np.inf

    # Calculate negative log likelihood
    nll = -np.sum(stats.logistic.logpdf(sample_data, loc=mean, scale=scale))

    return nll


# Initial parameter guess
initParams = [1, 1]

# Perform MLE using scipy.optimize.minimize
results = minimize(logistic_nll, initParams, method='Nelder-Mead')

# Extract estimated parameters
mu_est, s_est = results.x

print('Estimated Parameters:')
print(f'mean*  = {mu_est:.3f}, scale*  = {s_est:.3f}')
print('True Parameters:')
print(f'mean   = {mu:.3f}, scale   = {s:.3f}')

In [ ]:
import matplotlib.pyplot as plt

# Plot the logistic distribution fit
x = np.linspace(min(sample_data), max(sample_data), 100)
y = stats.logistic.pdf(x, loc=mu, scale=s)  # True logistic distribution

# Histogram of the sample data
num_bins = 20
plt.hist(sample_data, bins=num_bins, density=True,
         alpha=0.6, label="Sampled Data")

# Plot the true logistic distribution
plt.plot(x, y, label="True Logistic PDF", linewidth=2)

# Plot the estimated logistic distribution
y_est = stats.logistic.pdf(x, loc=mu_est, scale=s_est)
plt.plot(x, y_est, linestyle="dashed",
         label="Estimated Logistic PDF", linewidth=2)

plt.legend()
plt.xlabel("x")
plt.ylabel("Density")
plt.title("Logistic Distribution MLE Fit")
plt.show()

#### Answers

1. With more data, the law of large numbers ensures that the sample mean and variance converge to the true population parameters, leading to a more accurate MLE estimates for the mean and std.

2. If the initial guess is extremely far off, the optimizer will take longer to converge but also run the risk of getting stuck in a local minima, or fail. On the other hand, extremely close guesses will lead to faster convergence. The larger sample size the more robust the MLE is to poor initial guesses.

3. I assume that the stats.norm.logpdf function computes the log probability density of each point to get the log likelyhood, that can then be minimized in the scipy minimize function.